In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import random

# Set up the game.

Define the rank and suit of a card.

In [2]:
import enum

ranks = {
    "two" : 2,
    "three" : 3,
    "four" : 4,
    "five" : 5,
    "six" : 6,
    "seven" : 7,
    "eight" : 8,
    "nine" : 9,
    "ten" : 10,
    "jack" : 10,
    "queen" : 10,
    "king" : 10,
    "ace" : (1, 11)
}
    
class Suit(enum.Enum):
    spades = "spades"
    clubs = "clubs"
    diamonds = "diamonds"
    hearts = "hearts"

Define a card and a deck.

Implement shuffle, peek, & deal functions for the deck.

In [3]:
class Card:
    def __init__(self, suit, rank, value):
        self.suit = suit
        self.rank = rank
        self.value = value
        
    def __str__(self):
        return self.rank + " of " + self.suit.value

class Deck:
    def __init__(self, num=1):
        self.cards = []
        for i in range(num):
            for suit in Suit:
                for rank, value in ranks.items():
                    self.cards.append(Card(suit, rank, value))
                
    def shuffle(self):
        random.shuffle(self.cards)
        
    def deal(self):
        return self.cards.pop(0)
    
    def peek(self):
        if len(self.cards) > 0:
            return self.cards[0]
        
    def add_to_bottom(self, card):
        self.cards.append(card)
        
    def __str__(self):
        result = ""
        for card in self.cards:
            result += str(card) + "\n"
        return result
    
    def __len__(self):
        return len(self.cards)

# Set up Blackjack

### Define logic for evaluating the value of the dealer's hand.

Trickiest part is defining the logic for Aces.

_Dealer Logic will not change much! They must follow a set, predictable course of action._

In [4]:
# This follows the same, official rules every time.
# Still need to figure out what happens if there are multiple Aces.
def dealer_eval(player_hand):
    num_ace = 0
    use_one = 0
    for card in player_hand:
        if card.rank == "ace":
            num_ace += 1
            use_one += card.value[0] # use 1 for Ace
        else:
            use_one += card.value
    
    if num_ace > 0:
        # See if using 11 instead of 1 for the Aces gets the 
        # dealer's hand value closer to the [17, 21] range
        
        # The dealer will follow Hard 17 rules.
        # This means the dealer will not hit again if
        # the Ace yields a 17. 
        
        # This also means that Aces initially declared as 11's can
        # be changed to 1's as new cards come.
        
        ace_counter = 0
        while ace_counter < num_ace:
            # Only add by 10 b/c 1 is already added before
            use_eleven = use_one + 10 
            
            if use_eleven > 21:
                return use_one
            elif use_eleven >= 17 and use_eleven <= 21:
                return use_eleven
            else:
                # The case where even using Ace as eleven is less than 17.
                use_one = use_eleven
            
            ace_counter += 1
        
        return use_one
    else:
        return use_one

### Define logic for evaluating the value of the player's hand.

Trickiest part is defining the logic for Aces.

_This logic is subject to change._

In [5]:
def player_eval(player_hand):
    num_ace = 0
    # use_one means that every ace that in the hand is counted as one.
    use_one = 0
    for card in player_hand:
        if card.rank == "ace":
            num_ace += 1
            use_one += card.value[0] # use 1 for Ace
        else:
            use_one += card.value
    
    if num_ace > 0:
        # Define player policy for Aces:
        # Make Aces 11 if they get you to the range [18,21]
        # Otherwise, use one.
        
        ace_counter = 0
        while ace_counter < num_ace:
            # Only add by 10 b/c 1 is already added before
            use_eleven = use_one + 10 
            
            if use_eleven > 21:
                return use_one
            elif use_eleven >= 18 and use_eleven <= 21:
                return use_eleven
            else:
                # This allows for some Aces to be 11s, and others to be 1.
                use_one = use_eleven
            
            ace_counter += 1
        
        return use_one
    else:
        return use_one

### Define logic for the dealer's turn.

This will not change much since the dealer has to follow a defined protocol when making their moves.

In [6]:
def dealer_turn(dealer_hand, deck):
    # Calculate dealer hand's value.
    dealer_value = dealer_eval(dealer_hand)

    # Define dealer policy (is fixed to official rules)

    # The dealer keeps hitting until their total is 17 or more
    while dealer_value < 17:
        # hit
        dealer_hand.append(deck.deal())
        dealer_value = dealer_eval(dealer_hand)

    return dealer_value, dealer_hand, deck

## Define the OpenAI Gym Environment for Blackjack

In [70]:
import random
import numpy as np
import gym
from gym import error, spaces, utils
from gym.utils import seeding

INITIAL_BALANCE = 1000
ROUNDS_PER_EPISODE = 100
NUM_DECKS = 6

class BlackjackEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    
    def __init__(self):
        super(BlackjackEnv, self).__init__()
        
        self.reward_options = {"lose":-100, "tie":0, "win":100}
        
        # hit = 0, stand = 1
        self.action_space = spaces.Discrete(2)
        
        '''
        First element of tuple is the range of possible hand values for the player. (2 through 20)
        This is the possible range of values that the player will actually have to make a decision for.
        Any player hand value 21 or above already has automatic valuations, and needs no input from an
        AI Agent. 
        
        However, we also need to add all the hand values that the agent could possibly end up in when
        they bust. Maybe the agent can glean some correlations based on what hand value they bust at,
        so this should be in the observation space. Also, the layout of OpenAI Gym environment class
        makes us have to include the bust-value in the step() function because we need to return that
        done is true alongside the final obs, which is the bust-value.
        
        Therefore, we will also be adding the values 21 through 30 in our observation space. 30 is the
        highest possible hand value we can bust at. It happens when the agent hits at a hand value of 
        20 and gets a 10/J/Q/K. Getting an Ace in this position results in Blackjack for the agent
        because the Ace will just be used as a 1, not an 11.
        
        Therefore, the total observation space is a discrete range from 2 through 30. Therefore, the range
        is really 29 discrete numbers, and we will have to adjust the hand values to fit into the range
        1-29 by subtracting our hand value by 1.
        '''
        
        # Second element of the tuple is the range of possible values for the dealer's upcard. (1 through 11)
        self.observation_space = spaces.Tuple((spaces.Discrete(19), spaces.Discrete(11)))
        
        self.done = False
        
    def _take_action(self, action):
        if action == 0: # hit
            self.player_hand.append(self.bj_deck.deal())
            
        # define the value of the player's hand after any changes to the hand.
        self.player_value = player_eval(self.player_hand)
    
    def step(self, action):
        self._take_action(action)
        
        self.done = action == 1 or self.player_value >= 21
        
        rewards = 0
        if self.done:
            # CALCULATE REWARDS
            if self.player_value > 21: # above 21, player loses automatically.
                rewards = self.reward_options["lose"]
            elif self.player_value == 21: # blackjack! Player wins automatically.
                rewards = self.reward_options["win"]
            else:
                ## Begin dealer turn phase.

                dealer_value, self.dealer_hand, self.bj_deck = dealer_turn(self.dealer_hand, self.bj_deck)

                ## End of dealer turn phase

                #------------------------------------------------------------#

                ## Final Compare

                if dealer_value > 21: # dealer above 21, player wins automatically
                    rewards = self.reward_options["win"]
                elif dealer_value == 21: # dealer has blackjack, player loses automatically
                    rewards = self.reward_options["lose"]
                else: # dealer and player have values less than 21.
                    if self.player_value > dealer_value: # player closer to 21, player wins.
                        rewards = self.reward_options["win"]
                    elif self.player_value < dealer_value: # dealer closer to 21, dealer wins.
                        rewards = self.reward_options["lose"]
                    else:
                        rewards = self.reward_options["tie"]
        
        self.balance += rewards
        
        
        # Subtract by 1 to fit into the possible observation range.
        player_value_obs = self.player_value - 1
        
        upcard_value_obs = dealer_eval([self.dealer_upcard])
        
        obs = np.array([player_value_obs, upcard_value_obs])
        
        return obs, rewards, self.done, {}
    
    def reset(self): # resets game to an initial state
        # Our Blackjack deck will be made of 2 normal decks by default.
        self.bj_deck = Deck(NUM_DECKS)

        # Shuffle before beginning. Only shuffle once before the start of each game.
        self.bj_deck.shuffle()
         
        self.balance = INITIAL_BALANCE
        
        self.done = False
        
        # returns the start state for the agent
        # deal 2 cards to the agent and the dealer
        self.player_hand = [self.bj_deck.deal(), self.bj_deck.deal()]
        self.dealer_hand = [self.bj_deck.deal(), self.bj_deck.deal()]
        self.dealer_upcard = self.dealer_hand[0]
        
        # calculate the value of the agent's hand
        self.player_value = player_eval(self.player_hand)
        
        # Subtract by 1 to fit into the possible observation range.
        player_value_obs = self.player_value - 1
            
        upcard_value_obs = dealer_eval([self.dealer_upcard])
        
        obs = np.array([player_value_obs, upcard_value_obs])
        
        return obs
    
    def render(self, mode='human', close=False):
        # Render the environment to the screen
        
        hand_list = []
        for card in self.player_hand:
            hand_list.append(card.rank)
            
        upcard_value = dealer_eval([self.dealer_upcard])
        
        print(f'Balance: {self.balance}')
        print(f'Player Hand: {hand_list}')
        print(f'Player Value: {self.player_value}')
        print(f'Dealer Upcard: {upcard_value}')
        print(f'Done: {self.done}')
        
        print()

In [53]:
import random
env = BlackjackEnv()

total_rewards = 0
NUM_EPISODES = 1000

for _ in range(NUM_EPISODES):
    env.reset()

#     env.render()
#     print()
    while env.done == False:
        action = env.action_space.sample()
#         print("Action: ", action)

#         print()

        new_state, reward, done, desc = env.step(action)
        total_rewards += reward
#         env.render()
#         print()
        
avg_reward = total_rewards / NUM_EPISODES
print(avg_reward)

-41.1


## First-Visit Monte Carlo Algorithm

In [104]:
import numpy as np
import random

def get_Q_state_index(env, state):
    # the player value is already subtracted by 1 in the env when it returns the state.
    # subtract by 1 again to fit with the array indexing that starts at 0
    initial_player_value = state[0] - 1
    # dealer_upcard will be subtracted by 1 to fit with the array indexing that starts at 0
    dealer_upcard = state[1] - 1

    return (env.observation_space[1].n * (initial_player_value)) + (dealer_upcard)

def get_prob_of_best_action(env, state, Q, prob, epsilon):
    # Use the mapping function to figure out which index of Q corresponds to 
    # the player hand value + dealer upcard value that defines each state.
    Q_state_index = get_Q_state_index(env, state)
    
    # Use this index in the Q 2-D array to get a 2-element array that yield
    # the current Q-values for hitting (index 0) and standing (index 1) in this state.
    # Use the np.argmax() function to find the index of the action that yields the
    # rewards i.e. the best action we are looking for.
    best_action = np.argmax(Q[Q_state_index])
    
    # Retrieve the probability of the best action using the 
    # state/action pair as indices for the `prob` array,
    # which stores the probability of taking an action (hit or stand)
    # for a given state/action pair.
    return prob[Q_state_index][best_action] + 1 - epsilon
    
def update_prob_of_best_action(env, state, Q, prob, epsilon):

    Q_state_index = get_Q_state_index(env, state)
    
    best_action = np.argmax(Q[Q_state_index])
    
    # Slightly alter the probability of this best action being taken by using epsilon
    # Epsilon starts at 1.0, and slowly decays over time.
    # Therefore, as per the equation below, the AI agent will use the probability listed 
    # for the best action in the `prob` array during the beginning of the algorithm.
    # As time goes on, the likelihood that the best action is taken is increased from
    # what is listed in the `prob` array.
    # This allows for exploration of other moves in the beginning of the algorithm,
    # but exploitation later for a greater reward.
    prob[Q_state_index][best_action] = prob[Q_state_index][best_action] + 1 - epsilon
    
    other_action = 1 if best_action == 0 else 0
    prob[Q_state_index][other_action] = 1 - prob[Q_state_index][best_action]\
    
    return prob

def play_game(env, Q, epsilon, prob):
    # Can contain numerous state->action->reward tuples because a round of 
    # Blackjack is not always resolved in one turn.
    # However, there will be no state that has a player hand value that exceeds 20, since only initial
    # states BEFORE actions are made are used when storing state->action->reward tuples.
    episode = []
    
    state = env.reset()
    
    while env.done == False:
        if state[0] == 20: #Player was dealt Blackjack
            next_state, reward, env.done, info = env.step(1) # doesn't matter what action is taken.
            # don't do any episode analysis for this episode. This is a useless episode.
        else:
            Q_state_index = get_Q_state_index(env, state)
            best_action = np.argmax(Q[Q_state_index])
            prob_of_best_action = get_prob_of_best_action(env, state, Q, prob, epsilon)

            action_to_take = None

            if random.uniform(0,1) < prob_of_best_action: # Take the best action
                action_to_take = best_action
            else: # Take the other action
                action_to_take = 1 if best_action == 0 else 0

            next_state, reward, env.done, info = env.step(action_to_take)
            episode.append((state, action_to_take, reward))
            state = next_state
        
    return episode
    

def update_Q(env, episode, Q, alpha, gamma):
    '''
    THIS IS WHERE THE ALGORITHM HINGES ON BEING FIRST VISIT OR EVERY VISIT.
    I AM GOING TO USE FIRST-VISIT, AND HERE'S WHY.
    
    If you want first-visit, you need to use the cumulative reward of the entire
    episode when updating a Q-value for ALL of the state/action pairs in the
    episode, even the first state/action pair. In this algorithm, an episode
    is a round of Blackjack. Although the bulk of the reward may come from the
    2nd or 3rd decision, deciding to hit on the 1st decision is what enabled
    the future situations to even occur, so it is important to include the
    entire cumulative reward. We can reduce the impact of the rewards of the
    future decisions by lowering gamma, which will lower the G value for our
    early state/action pair in which we hit and did not get any immediate rewards.
    This will make our agent consider future rewards, and not just look at 
    each state in isolation despite having hit previously.
     
    If you want every visit, do not use the cumulative rewards when updating Q-values,
    and just use the immediate reward in this episode for each state/action pair.
    '''
    step = 0
    #env.render()
    for state, action, reward in episode:
        # calculate the cumulative reward of taking this action in this state.
        # Start from the immediate rewards, and use all the rewards from the
        # subsequent states. Do not use rewards from previous states.
        total_reward = 0
        gamma_exp = 0
        for step in range(step, len(episode)):
            total_reward += (gamma ** gamma_exp) * episode[step][2]
            gamma_exp += 1
    
#         print(state)
#         print(action)
#         print(reward)
#         print(total_reward)
#         print()
        
        # Update the Q-value
        Q_state_index = get_Q_state_index(env, state)
        Q[Q_state_index][action] = Q[Q_state_index][action] + alpha * (total_reward - Q[Q_state_index][action])
        
        
    return Q

def update_prob(env, episode, Q, prob, epsilon):
    for state, action, reward in episode:
        # Update the probabilities of the actions that can be taken given the current
        # state. The goal is that the new update in Q has changed what the best action
        # is, and epsilon will be used to create a small increase in the probability
        # that the new, better action is chosen.
        prob = update_prob_of_best_action(env, state, Q, prob, epsilon)
        
    return prob

def run_mc(env, num_episodes):
    '''
    observation_space[0] is the 29 possible player values.
    observation_space[1] is the 11 possible dealer upcards.

    Combining these together yields all possible states.

    Multiplying this with hit/stand yields all possible state/action pairs.

    This is the Q map.
    '''
    Q = np.zeros([env.observation_space[0].n * env.observation_space[1].n, env.action_space.n], dtype=np.float16)


    # This map contains the probability distributions for each action (hit or stand) given a state.
    # The state (combo of player hand value and dealer upcard value) index in this array yields a 2-element array
    # The 0th index of this 2-element array refers to the probability of "hit", and the 1st index is the probability of "stand"
    prob = np.zeros([env.observation_space[0].n * env.observation_space[1].n, env.action_space.n], dtype=np.float16) + 0.5

    # The learning rate. Very small to avoid making quick, large changes in our policy.
    alpha = 0.001

    epsilon = 1.0
    # The rate by which epsilon will decay over time.
    # Since the probability we take the option with the highest Q-value is 1-epsilon + probability,
    # this decay will make sure we are the taking the better option more often in the longrun.
    # This allows the algorithm to explore in the early stages, and exploit in the later stages.
    decay = 0.9999
    # The lowest value that epsilon can go to.
    # Although the decay seems slow, it actually grows exponentially, and this is magnified when
    # running thousands of episodes.
    epsilon_min = 0.1

    # may have to be tweaked later.
    gamma = 1.0
    
    for _ in range(num_episodes):
        epsilon = max(epsilon * decay, epsilon_min)
        
        episode = play_game(env, Q, epsilon, prob)
        
        Q = update_Q(env, episode, Q, alpha, gamma)
        prob = update_prob(env, episode, Q, prob, epsilon)
        
    # still need to write best_policy method
    return Q
        
env = BlackjackEnv()
new_Q = run_mc(env, 10000)
   
print(new_Q)

[[ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00]
 [ 1.0010e-01 -1.9995e-01]
 [ 2.9907e-01  0.0000e+00]
 [-9.9976e-02 -1.0010e-01]
 [ 1.9983e-01 -9.9854e-02]
 [-9.9976e-02 -2.0020e-01]
 [-9.9976e-02  0.0000e+00]
 [-1.9897e-01 -1.9983e-01]
 [-1.2434e-04 -9.9976e-02]
 [-1.9788e-01 -5.9766e-01]
 [-9.9976e-02 -9.9976e-02]
 [ 0.0000e+00  0.0000e+00]
 [ 9.8633e-02  0.0000e+00]
 [-2.9980e-01 -2.9907e-01]
 [-2.9907e-01  4.9019e-04]
 